In [1]:
from Systems import UnbalancedDisc
from Systems import NoisyUnbalancedDisc
from Systems import OutputUnbalancedDisc
import deepSI
from matplotlib import pyplot as plt
import numpy as np
#import autograd
import torch
import time
from torch import nn
#import quadprog as qp
import qpsolvers as qp

In [2]:
system = OutputUnbalancedDisc(dt=0.1, sigma_n=[0, 0])

class I_encoder(deepSI.fit_systems.SS_encoder):
    def __init__(self, nx = 2, na=2, nb=2, feedthrough=False) -> None:
        super().__init__(nx=nx, na=na, nb=nb, feedthrough=feedthrough)

    def init_nets(self, nu, ny): # a bit weird
        ny = ny if ny is not None else 1
        nu = nu if nu is not None else 1
        self.encoder = self.e_net(self.nb*nu+self.na*ny, self.nx, n_nodes_per_layer=self.e_n_nodes_per_layer, n_hidden_layers=self.e_n_hidden_layers, activation=self.e_activation)
        self.fn =      self.f_net(self.nx+nu,            self.nx, n_nodes_per_layer=self.f_n_nodes_per_layer, n_hidden_layers=self.f_n_hidden_layers, activation=self.f_activation)
        hn_in = self.nx + nu if self.feedthrough else self.nx
        self.hn =      nn.Identity(hn_in)#

I_enc = deepSI.load_system("systems/ObserverUnbalancedDisk_dt01_nab_4_e200")

In [3]:
# function that converts torch nn to casadi expression
from casadi import *

def CasADiHn(ss_enc, x):
    n_hidden_layers = 2#ss_enc.h_n_hidden_layers

    params = {}
    for name, param in ss_enc.hn.named_parameters():
        params[name] = param.detach().numpy()
    params_list = list(params.values())

    temp_nn = x
    for i in range(n_hidden_layers):
        W_NL = params_list[2+i*2]
        b_NL = params_list[3+i*2]
        temp_nn = mtimes(W_NL, temp_nn)+b_NL
        temp_nn = tanh(temp_nn)
    W_NL = params_list[2+n_hidden_layers*2]
    b_NL = params_list[3+n_hidden_layers*2]
    nn_NL = mtimes(W_NL, temp_nn)+b_NL

    W_Lin = params_list[0]
    b_Lin = params_list[1]
    nn_Lin = mtimes(W_Lin,x) + b_Lin

    return nn_NL + nn_Lin

def CasADiFn(ss_enc, x, u):
    n_hidden_layers = 2#ss_enc.f_n_hidden_layers

    params = {}
    for name, param in ss_enc.fn.named_parameters():
        params[name] = param.detach().numpy()
    params_list = list(params.values())
    
    xu = vertcat(x,u)

    temp_nn = xu
    for i in range(n_hidden_layers):
        W_NL = params_list[2+i*2]
        b_NL = params_list[3+i*2]
        temp_nn = mtimes(W_NL, temp_nn)+b_NL
        temp_nn = tanh(temp_nn)
    W_NL = params_list[2+n_hidden_layers*2]
    b_NL = params_list[3+n_hidden_layers*2]
    nn_NL = mtimes(W_NL, temp_nn)+b_NL

    W_Lin = params_list[0]
    b_Lin = params_list[1]
    nn_Lin = mtimes(W_Lin,xu) + b_Lin

    return nn_NL + nn_Lin

In [4]:
# declared sym variables
x = MX.sym("x",I_enc.nx,1)
nu = I_enc.nu if I_enc.nu is not None else 1
u = MX.sym("u",nu,1)

# convert torch nn to casadi function
rhs = CasADiFn(I_enc, x, u)
f = Function('f', [x, u], [rhs])

y_rhs = CasADiHn(I_enc, x)
h = Function('h', [x], [y_rhs])

# apply correction to casadi function such that fc(0) = 0
correction_f = f([0,0], 0)
rhs_c = rhs - correction_f
correction_h = h([0,0])
y_rhs_c = y_rhs - correction_h
f_c = Function('f_c', [x, u], [rhs_c])
h_c = Function('h_c', [x], [y_rhs_c])

In [5]:
np.array(correction_h)[:,0]

array([0.42514739])

In [6]:
# Box constraints
x_min = [-10, -10]
x_max = [10, 10]
u_min = -4
u_max = 4

# Initial and final values
x0 = [0,0]
x_ref = [0.001, 1.0]
u_ref = 0

# Weight matrices for the cost function
Q = np.array([[1,0],[0,100]])
R = 1

# MPC parameters
dt = 0.1
Nc = 4
Nsim = 50
#dlam = 0.01
stages = 100

In [7]:
# L = np.zeros([3*stages,1])
# lam = 0
dlam = 1.0/stages

# for i in range(stages):
#     L[i] = lam
#     L[i+stages] = lam + dlam/2
#     L[i+2*stages] = lam + dlam

#     lam = lam + dlam

n_states = np.shape(x)[0]
nx = n_states
n_controls = np.shape(u)[0]
nu = n_controls
ny = I_enc.ny if I_enc.ny is not None else 1

In [8]:
Jfx = Function("Jfx", [x, u], [jacobian(rhs_c,x)])
Jfu = Function("Jfu", [x, u], [jacobian(rhs_c,u)])
Jhx = Function("Jhx", [x, u], [jacobian(y_rhs_c,x)])

In [9]:
from lpv_int import *

[A, B, C] = lpv_int_C(x,nx,u,nu,ny,Jfx,Jfu,Jhx,dlam,stages)
get_A = Function("get_A",[x,u],[A])
get_B = Function("get_B",[x,u],[B])
get_C = Function("get_C",[x,u],[C])

In [10]:
Get_A = get_A.map(Nc, "thread", 32)
Get_B = get_B.map(Nc, "thread", 32)

In [11]:
from mpcUtil import *

In [17]:
f0 = np.array(correction_f.elements())[np.newaxis].T
f0

array([[0.06859763],
       [0.32200515]])

In [59]:
def getF0(list_A, f0, Nc, nx, nu):
    F0 = np.zeros([nx*Nc, nx])
    for i in range(Nc):
        for j in range(i+1):
            temp = np.eye(2)
            for l in range(j):
                print(str(i) + ", " + str(j) + ", " + str(l))
                temp = np.matmul(list_A[(nx*l):(nx*l+nx),:], temp)
            F0[i*nx:(i+1)*nx, :] = F0[i*nx:(i+1)*nx, :] + temp
    return F0

In [62]:
getF0(list_A, f0, 4, nx, nu) #np.kron(np.ones((3,1)), np.eye(2))

1, 1, 0
2, 1, 0
2, 2, 0
2, 2, 1
3, 1, 0
3, 2, 0
3, 2, 1
3, 3, 0
3, 3, 1
3, 3, 2


array([[ 1.,  0.],
       [ 0.,  1.],
       [ 3.,  0.],
       [ 0.,  3.],
       [ 7.,  0.],
       [ 0.,  7.],
       [15.,  0.],
       [ 0., 15.]])

In [37]:
list_A

array([[ 0.21917246,  0.26985018],
       [-1.03306645,  0.51417798],
       [ 0.36560131,  0.34343052],
       [-1.3875607 ,  0.45466903],
       [ 0.27050238,  0.36154198],
       [-1.50674346,  0.53530089],
       [ 0.35240265,  0.39804248],
       [-1.51583822,  0.55563059]])

In [12]:
# logging list
x0 = np.array([0.0,0.5])
u_log = np.zeros(Nsim*n_controls)
x_log = np.zeros((Nsim+1)*n_states)
x_log[:nx] = x0
components_times = np.zeros(3) # getAB, solve, denorm and sim
component_start = 0
t = np.zeros(Nsim)
t0 = 0
comp_t_log = np.zeros(Nsim)
start = time.time()
lpv_counter = np.zeros(Nsim,int)

# normalize reference list
norm = I_enc.norm
#reference_list_normalized = (reference_list - norm.y0[1])/norm.ystd[1]
x0_norm = (x0 - norm.y0)/norm.ystd

# set initial values for x
x = np.tile(x0_norm, Nc)
u = np.zeros(Nc*nu)

list_A = np.zeros([Nc*nx, nx])
list_B = np.zeros([Nc*nx, nu])
list_A_p = np.zeros([Nc*nx, nx])
list_B_p = np.zeros([Nc*nx, nu])
# pA = np.zeros([Nc*nx, nx])
# pB = np.zeros([Nc*nx, nu])
Psi = getPsi(Nc, R)
Omega = getPsi(Nc, Q)
D, E, M, c = getDEMc(x_min, x_max, u_min, u_max, Nc, nx, nu)

ne = 1
Ge = np.zeros((Nc+ne, Nc+ne))

for mpciter in range(5):
    start_time_iter = time.time()
    
    while True:
        # for i in np.arange(Nc):
        #     list_A[(n_states*i):(n_states*i+n_states),:] = get_A(x[i*nx:(i+1)*nx],u[i*nu:(i+1)*nu])
        #     list_B[(n_states*i):(n_states*i+n_states),:] = get_B(x[i*nx:(i+1)*nx],u[i*nu:(i+1)*nu])
        
        pA = Get_A(np.vstack(np.split(x,Nc)).T,u)
        for i in range(Nc):
            list_A[(n_states*i):(n_states*i+n_states),:] = pA[:,i*nx:(i+1)*nx]
        pB = Get_B(np.vstack(np.split(x,Nc)).T,u)
        for i in range(Nc):
            list_B[(n_states*i):(n_states*i+n_states),:] = pB[:,i*nu:(i+1)*nu]

        Phi = getPhi(list_A, Nc, nx, nu)
        Gamma = getGamma(list_A, list_B, Nc, nx, nu)
        G = 2*(Psi+(Gamma.T@Omega@Gamma))
        F = 2*(Gamma.T@Omega@Phi)
        L = (M@Gamma) + E
        W = -D - (M@Phi)

        Le = np.hstack((L, -np.ones((Nc*2*(nx+nu)+2*nx,1))))
        Ge[:Nc, :Nc] = G
        Ge[Nc:,Nc:] = 100
        Fe = np.hstack((F@x[:2], np.zeros(ne)))
        

        u_old = u
        #x_ss = x[:2] - [0, 1]
        #u = -np.linalg.inv(G)@F@x[:2]
        #u = qp.solve_qp(G,F@x[:2],L,(W@x[:2]) + c[:,0], solver="quadprog")
        ue = qp.solve_qp(Ge,Fe,Le,(W@x[:2]) + c[:,0], solver="quadprog")
        u = ue[:Nc]
         
        x[nx:Nc*nx] = ((Phi@x[:2]) + Gamma@u)[:(Nc-1)*nx]# + np.tile(np.array(correction_f)[:,0], (Nc-1))
        
        lpv_counter[mpciter] += 1
        if (lpv_counter[mpciter] >= 5) or (np.linalg.norm(u-u_old) < 1e-7):
            break

    print("MPC iteration: ", mpciter+1)
    print("LPV counter: ", lpv_counter[mpciter])
    
    t[mpciter] = t0
    t0 = t0 + dt
    
    # denormalize x and u
    x_denormalized = norm.ystd*x0_norm + norm.y0
    u_denormalized = norm.ustd*u[0] + norm.u0

    # make system step and normalize
    x_denormalized = system.f(x_denormalized, u_denormalized)
    x_measured = system.h(x_denormalized, u_denormalized)
    x0_norm = (x_measured - norm.y0)/norm.ystd

    x_log[(mpciter+1)*nx:(mpciter+2)*nx] = x_measured
    u_log[mpciter] = u_denormalized
    
    x = np.hstack((x[nx:(Nc+1)*nx],x[-2:]))
    x[:nx] = x0_norm
    u = np.hstack((u[nx:Nc*nx],u[-2:]))

    # finished mpc time measurement
    end_time_iter = time.time()
    comp_t_log[mpciter] = end_time_iter - start_time_iter

end = time.time()
runtime = end - start
print(runtime)

MPC iteration:  1
LPV counter:  5
MPC iteration:  2
LPV counter:  5


TypeError: iteration over a 0-d array

In [ ]:
x_reference_list = np.load("references/randomLevelTime15_20Range-1_1Nsim500.npy")
x_reference_list_normalized = ((x_reference_list.T - norm.y0)/norm.ystd).T
y_reference_list = x_reference_list_normalized[1,:]
#plt.plot(y_reference_list)

In [ ]:
def getXsUs_Cs(y_reference_list_normalize, nx, nu, ny, Nsim, u_min, u_max, x_min, x_max, get_A, get_B, C, f0, h0):
    ne = 1 #number of variables in epsilon
    Q = np.eye(ny) # add this as variable of function
    R = np.eye(nu) # add this as variable of function
    lam = 1000
    
    In = np.eye(nx)
    Im = np.eye(nu)
    Zn = np.zeros((nu,nx))
    Zm = np.zeros((nx,nu))

    Mi = np.vstack((Zn, Zn, -In, In))
    Ei = np.vstack((-Im, Im, Zm, Zm))
    h = np.array([list(itertools.chain([-u_min, u_max], [x*-1 for x in x_min],  x_max))]).T

    T = np.zeros((2*(nx+nu), nx+nu+ne))
    T[:,:nx] = Mi
    T[:,nx:nx+nu] = Ei
    T[:,nx+nu:] = -np.ones((2*(nx+nu),ne))

    b = np.zeros((nx+ny, 1))
    b[:nx] = f0

    P = np.zeros((nx+nu+ne, nx+nu+ne))
    #P[:nx, :nx] = C.T@Q@C
    P[nx:nx+nu, nx:nx+nu] = R
    P[nx+nu:, nx+nu:] = lam

    q = np.zeros((nx+nu+ne,1))
    
    xs = np.zeros(nx)
    us = np.zeros(nu)
    xue = np.zeros(nx+nu+ne)
    As = np.zeros((nx,nx))
    Bs = np.zeros((nx,nu))
    Cs = np.zeros((ny,nx))

    A = np.zeros((nx+ny, nx+nu+ne))
    #A[nx:nx+ny,:nx] = C #change this to getC from xs us when needed

    x_reference_list_normalized = np.zeros((nx, Nsim))
    u_reference_list_normalized = np.zeros((nu, Nsim))
    e_reference_list_normalized = np.zeros((ne, Nsim))

    for j in range(Nsim):

        b[nx:nx+ny] = y_reference_list_normalize[j] - h0 #+ correction_h #add h0 here when needed
        q[:nx,0] = C.T@Q@(h0 - y_reference_list_normalize[j])

        for i in range(20):
            As[:,:] = get_A(xs, us)
            Bs[:,:] = get_B(xs, us)
            Cs[:,:] = get_C(xs, us)

            A[:nx,:nx] = np.eye(nx) - As
            A[:nx,nx:nx+nu] = -Bs
            A[nx:nx+ny,:nx] = Cs
            q[:nx,0] = Cs.T@Q@(h0 - y_reference_list_normalize[j])
            P[:nx, :nx] = Cs.T@Q@Cs

            #xu[:] = (np.linalg.inv(A)@b)[:,0]
            xue[:] = (qp.solve_qp(P,q,T,h[:,0],A,b[:,0],solver="osqp"))

            xold = xs
            uold = us
            xs = xue[:nx]
            us = xue[nx:nx+nu]
            e = xue[nx+nu:]

            if np.linalg.norm(xs-xold) <= 1e-6 and np.linalg.norm(us-uold) <= 1e-6:
                break

        x_reference_list_normalized[:,j] = xs
        u_reference_list_normalized[:,j] = us
        e_reference_list_normalized[:,j] = e
        
    return x_reference_list_normalized, u_reference_list_normalized, e_reference_list_normalized

In [ ]:
C = np.array([[0, 1]])
ny = 1
h0 = np.zeros(1)
Nsim = 100
x_reference_list_normalized, u_reference_list_normalized, e_reference_list_normalized = getXsUs(y_reference_list, nx, nu, ny, Nsim, \
    u_min, u_max, x_min, x_max, get_A, get_B, C, correction_f, h0)

In [ ]:
Xs = np.reshape(x_reference_list_normalized[:,1:Nc+1].T, (2*Nc,1))
xs = x_reference_list_normalized[:,:1]
Us = u_reference_list_normalized[:,:Nc].T
Fs = 2*(Gamma.T@Omega@(Phi@(x[:2][np.newaxis].T) + Xs) + Psi.T@Us)
Fs.shape, F.shape

In [ ]:
plt.plot(x_reference_list_normalized[1,:])

In [ ]:
plt.plot(e_reference_list_normalized[0,:])

In [ ]:
#x_complete = np.hstack((x_log))

plt.subplot(2,3,1)
plt.plot(x_log[0::nx])

plt.subplot(2,3,2)
plt.plot(x_log[1::nx])

plt.subplot(2,3,3)
plt.plot(u[:])

plt.subplot(2,3,4)
plt.plot(comp_t_log[:])